# Facial Expression Classifier

In [8]:
#Dataset -> Facial expressions Dataset from Kaggle
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#lol

Num GPUs Available:  0


In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential 
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [6]:
#generating dataset ,using keras.utils.image_dataset_from_directory,from my folders

train_dataset=keras.utils.image_dataset_from_directory(
    directory='./images/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)

    
)
print("Label names:", train_dataset.class_names)


Found 28821 files belonging to 7 classes.


2025-05-29 22:35:38.328607: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Label names: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [9]:
#creating validation dataset

In [13]:
validation_dataset=keras.utils.image_dataset_from_directory(
directory='./images/validation/',
labels='inferred',
label_mode='int',
batch_size=32,
image_size=(256,256)
    
)

Found 7066 files belonging to 7 classes.


In [15]:
#normalizing the data :
#from 0-255 to 0-1

In [17]:
def process(image,label):
    image=tf.cast(image/255. ,tf.float32)
    return image,label
train_dataset=train_dataset.map(process)

    

In [18]:
validation_dataset=validation_dataset.map(process)


# Model Creation

In [24]:
from tensorflow.keras.layers import Input


In [27]:
model=Sequential()
model.add(Input(shape=(256, 256, 3)))
model.add(Conv2D(32,kernel_size=(3,3),padding="valid",activation="relu",input_shape=(256,256,3)))
#here 32,kernel size is ig default ,lets change them later,padding valid ,means
#not focusing on edges .but more focus on center as no padding,activation =
#relu ->good for non linear data
#256,256, 3->RGB channels
model.add(BatchNormalization()) # helps in avoiding local minima ,speed up,model more stable
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="valid"))

#2ndconv layer
model.add(Conv2D(64,kernel_size=(3,3),padding="valid",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="valid"))

#3rd layer
model.add(Conv2D(128,kernel_size=(3,3),padding="valid",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="valid"))

#flattening for ann
model.add(Flatten()) #2D -> 1D ,for ann

model.add(Dense(128,activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(7,activation="softmax"))

In [28]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 254, 254, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 125, 125, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 60, 60, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,583 (56.64 MB)

 Trainable params: 14,848,135 (56.64 MB)

 Non-trainable params: 448 (1.75 KB)

In [29]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

#using sparse categorical crossentropy as its goo for multi class classification

In [31]:
history=model.fit(train_dataset,epoch=10,validation_data=validation_dataset)

TypeError: TensorFlowTrainer.fit() got an unexpected keyword argument 'epoch'